# <center> Endurance Direct Fraud PoC </center>

## Notebook setup

In [1]:
!pip install pyneoinstance boto3 --quiet

In [2]:
import os
import re
import string
import warnings
import pandas as pd
import numpy as np
from typing import Tuple
from pyneoinstance import Neo4jInstance
from pyneoinstance.fileload import load_yaml_file

warnings.filterwarnings('ignore')

### Naming Conventions & Recommendations

In [3]:
# Follow these best practices 
# https://neo4j.com/docs/cypher-manual/current/syntax/naming/

## Utility Functions

In [4]:
def get_parallel_info(num_rows: int) -> Tuple[int,bool]:
    partitions = 1
    parallel = True
    batches = int(num_rows/1000)
    if batches > 1:
        partitions = batches
        parallel = True
    return (partitions, parallel)

In [38]:
config_file = load_yaml_file('cypher.yaml')
neo_info = config_file['neo4j']
pre_load_queries = config_file['queries']['pre-load']
node_queries = config_file['queries']['load']['nodes']
rela_queries = config_file['queries']['load']['relationships']
#post_load_queries = config_file['queries']['post-load']
node_labels = list(node_queries.keys())
rela_types = list(rela_queries.keys())

#### Claim notes

In [8]:
filtered = df_claim_nts[df_claim_nts['iEntityType'].notnull()]
filtered.head()

,iId,iClaimId,sNoteType,sText,bSystemNote,iInsertedUserId,dtInsertedTime,iUpdatedUserId,dtUpdatedTime,sEnteredByUserName,sUpdatedByUserName,dtReminderActive,dtReminderIssued,sReminderSentBy,iExpectedOdom,iEntityType,iAssignedToUserId,bExternal
939390,27390558,4908272,0,"CCC for Failure#1:\nComplaint:\nCS every time the vehicle is started there is a buzzing noise that goes away after he reaches 25 MPH\n\nCause:\nthe buzzing noise is coming from the water pump, it is leaking from the gasket, coolant level was at the max and in looked like it could be changed, NO signs of damage, rust or corrosion, mods, or commercial use\n\nCorrection:\nReplace water pump\n\nCCC for Failure#2:\nComplaint:\nTF\n\nCause:\nboth outer tie rod ends have play and the boots are split, NO signs of damage\n\nCorrection:\nReplace both outer tie rods\n\nCCC for Failure#3:\nComplaint:\nTF\n\nCause:\ncompressor is not engaging, does have good power and ground going to it, freon was full, no signs of damage or rust or corrosion\n\nCorrection:\nReplace a/c compressor\n\nCCC for Failure#4:\nComplaint:\nTF\n\nCause:\nfound 2 TPM sensor have failed, both the rear sensors are shorted, NO signs of damage\n\nCorrection:\nReplace 2 TPM sensors\n\nCCC for Failure#5:\nComplaint:\nTF\n\nCause:\ndrive belt is frayed/worn\n\nCorrection:\nReplace drive belt\n\n",True,83141.00,2022-04-02 12:10:20.437000,83141,2022-04-02 12:10:20.437000,ALEX VALLES,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN
939391,27390559,4908272,0,"New Claim Created. Loss Odometer: 96,051, Loss Date: 04/02/2022",True,83141.00,2022-04-02 12:10:20.437000,83141,2022-04-02 12:10:20.437000,ALEX VALLES,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN
939392,27390568,4907723,0,shop cb - states ch states we are now covering claim = sent message to james d for view,False,0.00,2022-04-02 12:14:10.537000,0,2022-04-02 12:14:10.537000,Dean Melchiorre,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN
939393,27390576,4908226,0,*** CLAIM RESOLUTION EVENT ***\n \nTOTAL PAYABLE TO RF:\n0.00\n \nLINES AUTHORIZED:\nn/a\n \nLINES DENIED:\nno1-2\n \nREASON FOR DENIAL:\npart not listed for coverage,False,0.00,2022-04-02 12:16:50.797000,0,2022-04-02 12:16:50.797000,Shaun Ivory,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN
939394,27390577,4908226,0,CCC for Failure#1:\nComplaint:\nAdvised us that he needed an alternator.\n\nCause:\nBattery failed load test.\nP0031\nAF Ratio sensor not fluctuating\npower and ground good to sensor\n\nCorrection:\nReplace Battery. Alternator needs to be re-checked once battery is replaced.\nAF Ratio\n\nCCC for Failure#2:\nComplaint:\ninspection\n\nCause:\nps belt worn out\nbelt tensioner making noise\nidler pulley making noise\n\nCorrection:\ndrive belt tensioner\nidler pulley\n\n,True,78905.00,2022-04-02 12:16:56.870000,78905,2022-04-02 12:16:56.870000,Shaun Ivory,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN


In [9]:
df_claim_nts.head()

,iId,iClaimId,sNoteType,sText,bSystemNote,iInsertedUserId,dtInsertedTime,iUpdatedUserId,dtUpdatedTime,sEnteredByUserName,sUpdatedByUserName,dtReminderActive,dtReminderIssued,sReminderSentBy,iExpectedOdom,iEntityType,iAssignedToUserId,bExternal
0,22470440,4247341,0,Claim status is changed from Open To Paid.,False,0.00,2019-07-05 07:13:00,0,2020-08-30 16:43:08.420000,BATCHIMPORT,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
1,22470441,4254429,0,The inspection request has been received Request ID 2739962.,False,0.00,2018-10-05 10:17:00,0,2020-08-30 16:43:08.420000,scott.davis,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
2,22470442,4253399,0,Claim is created.,False,0.00,2018-01-05 11:04:00,0,2020-08-30 16:43:08.420000,robert.zadlo,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
3,22470443,4257884,0,Awaiting AmTrust email HUD David P,False,0.00,2017-09-27 10:14:00,0,2020-08-30 16:43:08.420000,monique.rivera,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
4,22470444,4246217,0,"I past history note stated :Reviewed start mileage with Rob V, Ok to proceed with claim . from 07/03/18",False,0.00,2018-11-07 11:00:00,0,2020-08-30 16:43:08.420000,giovanni.fioccola,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN


In [10]:
 df_claim_nts['iInsertedUserId'].fillna(value=-1, inplace=True)
 df_claim_nts['iAssignedToUserId'].fillna(value=-1, inplace=True)

In [11]:
df_claim_nts['iInsertedUserId'] = df_claim_nts['iInsertedUserId'].astype(int)
df_claim_nts['iAssignedToUserId'] = df_claim_nts['iAssignedToUserId'].astype(int)

#### Claim Header

In [12]:
df_claim_hd.head()

,iId,iContractId,sClaimNumber,sRONumber,sClaimStatusCode,sClaimStatusDesc,dRODate,dClaimDate,iServiceOdom,sPriorityCode,sActivityCode,nActualCashValue,iAssignedToId,sAssignedToFirstName,sAssignedToLastName,dtInserted,dtUpdated,iInsertedUserId,iUpdatedUserId,btCommunicationPreferenceSpecialHandling,iSpecialHandlingServicerId,sSpecialHandlingEntityType,dtEstimatedCompletionDate,sClaimRefNumber
0,4205674,13898388,E_C000009290,NaN,D,Denied,2015-06-03,2015-06-03,96080,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-30 12:40:04.940000,2020-08-30 12:40:04.940000,57535,NaN,NaN,NaN,NaN,NaN,NaN
1,4205675,13871780,E_C000022571,57821,X,Paid,2016-08-22,2016-08-22,87393,NaN,103,NaN,NaN,NaN,NaN,2020-08-30 12:40:04.940000,2020-08-30 12:40:04.940000,57535,NaN,NaN,NaN,NaN,NaN,NaN
2,4205676,13871780,E_C000018718,55873,X,Paid,2016-05-23,2016-05-23,73666,NaN,113,NaN,NaN,NaN,NaN,2020-08-30 12:40:04.940000,2020-08-30 12:40:04.940000,57535,NaN,NaN,NaN,NaN,NaN,NaN
3,4205677,13871851,E_C000024345,405764,W,Closed,2016-10-03,2016-10-03,39499,NaN,103,NaN,NaN,NaN,NaN,2020-08-30 12:40:04.940000,2020-08-30 12:40:04.940000,57535,NaN,NaN,NaN,NaN,NaN,NaN
4,4205678,13871839,E_C000024106,74169,X,Paid,2016-09-27,2016-09-27,51410,NaN,103,NaN,NaN,NaN,NaN,2020-08-30 12:40:04.940000,2020-08-30 12:40:04.940000,57535,NaN,NaN,NaN,NaN,NaN,NaN


#### Claim Details

In [13]:
filtered = df_claim_det[df_claim_det['iClaimPayeeId'] == 4366220]
filtered

,iId,iClaimId,iClaimCompontentId,iClaimPayeeId,sDetailStatusCode,sDetailStatusDesc,sDetailTypeCode,iStatusReasonId,sReasonCode,sReasonDescription,sDetailTypeDescription,iLossCodeId,sLossCode,sLossCodeDescription,bSubletFlag,btGoodWill,iWarrantyMonth,iWarrantyOdom,sPartNumber,sPartDescription,nPartReqQty,nPartReqUnitCost,nPartAuthQty,nPartAuthUnitCost,iTireTread,nPartSubmitQty,cPartSubmitTotal,nLaborSubmitQty,cLaborSubmitTotal,cPartTax1,cPartTax2,cLaborTax1,cLaborTax2,nLaborReqQty,nLaborReqUnitCost,nLaborAuthQty,nLaborAuthUnitCost,sPayMethod,dAuthorizedDate,sAuthNumber,dDeniedDate,dApprovedToPayDate,dtPaymentProcessed,cTotalPending,cTotalDenied,cTotalPaid,sPaymentInstructCode,dtInserted,dtUpdated,bSelfAuthorized,iInsertedUserId,iUpdatedUserId,iRefPaymentId,sLaborOpCode,iAuthorizedUserId,iDeniedUserId
135902,10709622,4205675,5648324,4366220,X,Paid,D,NaN,NaN,NaN,Deductible,82569,12ST142VSC,Steering Shaft,False,NaN,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,-100.00,0.00,0.00,WEX,NaN,119706.00,NaN,NaN,2016-09-09 00:00:00,0.00,0.00,0.00,NaN,2020-08-30 14:52:44.600000,NaN,False,57535,NaN,0,NaN,NaN,NaN
159398,10733118,4205675,5648324,4366220,X,Paid,J,NaN,NaN,NaN,Job,82569,12ST142VSC,Steering Shaft,False,NaN,0.00,0.00,484701ma0b,NaN,1.00,203.95,1.00,164.11,0,1.00,164.11,0.00,0.00,9.85,0.00,0.00,0.00,0.00,0.00,0.00,0.00,WEX,NaN,119713.00,NaN,NaN,2016-09-09 00:00:00,0.00,0.00,173.96,NaN,2020-08-30 14:52:44.600000,NaN,False,57535,NaN,0,NaN,NaN,NaN
159399,10733119,4205675,5648324,4366220,X,Paid,J,NaN,NaN,NaN,Job,82569,12ST142VSC,Steering Shaft,False,NaN,0.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00,0,0.00,0.00,3.00,345.00,0.00,0.00,20.70,0.00,3.00,115.00,3.00,115.00,WEX,NaN,119715.00,NaN,NaN,2016-09-09 00:00:00,0.00,0.00,365.70,NaN,2020-08-30 14:52:44.600000,NaN,False,57535,NaN,0,NaN,NaN,NaN
160072,10733792,4205675,5648324,4366220,X,Paid,J,NaN,NaN,NaN,Job,82569,12ST142VSC,Steering Shaft,False,NaN,0.00,0.00,488101ma6d,NaN,1.00,2626.45,1.00,2624.45,0,1.00,2624.45,0.00,0.00,157.47,0.00,0.00,0.00,0.00,0.00,0.00,0.00,WEX,NaN,119714.00,NaN,NaN,2016-09-09 00:00:00,0.00,0.00,2781.92,NaN,2020-08-30 14:52:44.600000,NaN,False,57535,NaN,0,NaN,NaN,NaN


#### Claim Payees

In [14]:
df_claim_pay.head()

,iId,iClaimId,bDefaultPayee,bDefaultServicer,sPayeeType,sPayeeNumber,sPayeeName,sPayeeContactFirstName,sPayeeContactLastName,sPayeeAddress1,sPayeeAddress2,sPayeeCity,sPayeeState,sPayeeZipCode,sPayeePhoneNumber,sPayeeEmail,sPayMethod,sCreditCardNotifyMethod,cLaborRate,nPartTax1,nPartTax2,nLaborTax1,nLaborTax2,dtInserted,dtUpdated,iInsertedUserId,iUpdatedUserId
0,4249148,4295624,False,False,Servicer,CENTRO,Centro Inspection Agency,NaN,NaN,P.O. Box 309,NaN,Lincroft,NJ,07738,8004773607.00,jaroschak@centroinspection.com,CHK,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
1,4249149,4282403,True,True,Servicer,SC010663,Bryan Honda,NaN,NaN,4104 Raeford Rd,NaN,Fayetville,NC,28309,9104831234.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
2,4249150,4234318,True,True,Servicer,SC002279,Momentum VW,NaN,NaN,19550 NW Freeway,NaN,Jersey Village,TX,77065,2819255100.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
3,4249151,4207583,True,True,Seller,262,Napleton's Mid Rivers CDJR,NaN,NaN,4951 Veterans Memorial Pkwy,NaN,Saint Peters,MO,63376,6369288000.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
4,4249152,4285530,False,False,Servicer,CENTRO,Centro Inspection Agency,NaN,NaN,P.O. Box 309,NaN,Lincroft,NJ,07738,8004773607.00,jaroschak@centroinspection.com,CHK,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN


In [15]:
filtered = df_claim_pay[df_claim_pay['sPayeeAddress1'].str.contains('15301', na=False)]
filtered.head()

,iId,iClaimId,bDefaultPayee,bDefaultServicer,sPayeeType,sPayeeNumber,sPayeeName,sPayeeContactFirstName,sPayeeContactLastName,sPayeeAddress1,sPayeeAddress2,sPayeeCity,sPayeeState,sPayeeZipCode,sPayeePhoneNumber,sPayeeEmail,sPayMethod,sCreditCardNotifyMethod,cLaborRate,nPartTax1,nPartTax2,nLaborTax1,nLaborTax2,dtInserted,dtUpdated,iInsertedUserId,iUpdatedUserId
44,4249192,4263230,True,True,Servicer,SC010094,Truck City Ford,NaN,NaN,15301 S IH 35,NaN,Buda,TX,78610,5123120800.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
16241,4265389,4216722,True,True,Servicer,SC010094,Truck City Ford,NaN,NaN,15301 S IH 35,NaN,Buda,TX,78610,5123120800.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
24837,4273985,4265278,True,True,Servicer,SC010094,Truck City Ford,NaN,NaN,15301 S IH 35,NaN,Buda,TX,78610,5123120800.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
36337,4285485,4297544,True,True,Servicer,SC010094,Truck City Ford,NaN,NaN,15301 S IH 35,NaN,Buda,TX,78610,5123120800.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN
46380,4295533,4268242,True,True,Servicer,SC010094,Leief Johnson Truck City Ford,NaN,NaN,15301 S IH 35,NaN,Buda,TX,78610,5123120800.00,NaN,WEX,None,0.00,0.00,0.00,0.00,0.00,2020-08-30 12:56:36.220000,NaN,57535,NaN


In [16]:
df_security['iOwnerId'].unique()

array([0.00000e+00, 5.60000e+01,         nan, ..., 2.25723e+05,
       2.25727e+05, 2.25732e+05])

In [17]:
a = df_security[df_security['sUserFirstName'].notnull()]
a.head()

,iid,sUserFirstName,sUserLastName,iUserTypeId,iOwnerId,sUserEmail,sUserStatusCode,iTemplateId,sUserTemplateDesc,dtInsertedTime,iInsertedUserId,iUpdatedUserId,dtUpdatedTime
19,57535,ENDR,User,9,56.00,endr@pcmicorp.com,A,0.00,NaN,2019-10-21 02:31:01.847000,0,57535,2024-09-24 15:32:26.080000
20,57582,David,Sulfridge,9,56.00,david.sulfridge@endurancedirect.com,I,57533.00,NaN,2019-10-22 09:17:45.207000,57535,81543,2022-02-17 14:04:39.230000
21,57626,Alex,Agran,9,56.00,alex.agran@endurancedirect.com,A,0.00,NaN,2019-10-23 13:54:57.300000,57535,57626,2024-09-26 08:54:31.080000
22,57627,Cynthia,Becerril,9,56.00,cynthiab@endurancedirect.com,A,57533.00,NaN,2019-10-23 13:56:54.410000,57535,57627,2024-09-26 15:01:34.450000
23,57628,Ernesto,Cano,9,56.00,ernestoc@enduranceds.com,I,57533.00,Ernesto Cano,2019-10-23 13:57:51.390000,57535,81543,2023-12-07 16:45:38.910000


In [18]:
df_security.head()

,iid,sUserFirstName,sUserLastName,iUserTypeId,iOwnerId,sUserEmail,sUserStatusCode,iTemplateId,sUserTemplateDesc,dtInsertedTime,iInsertedUserId,iUpdatedUserId,dtUpdatedTime
0,57516,NaN,NaN,1,0.00,NaN,A,0.00,Agent,2019-10-21 02:31:01.110000,0,87591,2024-09-12 16:08:22.493000
1,57517,NaN,NaN,9,56.00,NaN,A,0.00,Claims CS,2019-10-21 02:31:01.110000,0,57646,2021-10-04 12:34:44.947000
2,57518,NaN,NaN,9,56.00,NaN,A,0.00,Claim Under Adjuster,2019-10-21 02:31:01.110000,0,60880,2024-03-22 13:47:29.840000
3,57519,NaN,NaN,9,56.00,NaN,A,0.00,Claims Manager,2019-10-21 02:31:01.110000,0,57646,2024-08-30 14:42:52.940000
4,57520,NaN,NaN,9,56.00,NaN,A,0.00,ClaimsProcessor,2019-10-21 02:31:01.110000,0,62700,2022-06-08 15:35:03.957000


In [19]:
df_security['iOwnerId'].fillna(value=-1, inplace=True)

In [20]:
df_security['iOwnerId'] = df_security['iOwnerId'].astype(int)

In [27]:
df_vin.head()

,Vin,BackfillBulkInsertGuid,ContractDate,Vehicle,Date,Price,Mileage,SellerName,SellerAddress,SellerCity,SellerState,SellerZip,SellerWebsite,SellerPhone,CreatedDate,UpdatedDate,CreatedBy,UpdatedBy,IsActive
0,19UDE2F30GA021226,00000000-0000-0000-0000-000000000000,2019-04-27 00:00:00+00:00,Acura ILX 2016,2024-09-08 00:00:00+00:00,13995.00,105466.00,Hilario Auto Sales,145 Chandler St,Worcester,MA,1609,https://www.hilarioauto.com,508-980-5183,2024-09-30 09:00:09.497667+00:00,2024-09-30 09:00:09.497668+00:00,system,system,False
1,19UDE2F30LA000497,00000000-0000-0000-0000-000000000000,2020-10-08 00:00:00+00:00,Acura ILX 2020,2024-09-06 00:00:00+00:00,18900.00,64987.00,Doral Acura,10455 NW 12th St,Miami,FL,NaN,www.doralacura.com,786-686-8791,2024-09-30 09:00:09.497684+00:00,2024-09-30 09:00:09.497684+00:00,system,system,False
2,19UDE2F31HA010902,00000000-0000-0000-0000-000000000000,2018-09-24 00:00:00+00:00,Acura ILX 2017,2024-09-08 00:00:00+00:00,15864.00,84058.00,Swickard Buick Gmc Of Thousand Oaks,NaN,NaN,NaN,NaN,cadillacthousandoaks.com,NaN,2024-09-30 09:00:09.497687+00:00,2024-09-30 09:00:09.497687+00:00,system,system,False
3,19UDE2F34HA004415,00000000-0000-0000-0000-000000000000,2023-11-08 00:00:00+00:00,Acura ILX 2017,2024-08-30 00:00:00+00:00,16888.00,82193.00,Brannon Honda,300 Gadsden Hwy,Birmingham,AL,35235,https://www.brannonhonda.com,205-833-1233,2024-09-30 09:00:09.497673+00:00,2024-09-30 09:00:09.497673+00:00,system,system,False
4,19UDE2F34JA008924,00000000-0000-0000-0000-000000000000,2020-06-22 00:00:00+00:00,Acura ILX 2018,2024-03-06 00:00:00+00:00,19995.00,67009.00,Big Island Honda,124 Wiwoole St,Hilo,HI,96720,tonyhondahilo.com,808-961-5505,2024-09-30 09:00:09.495180+00:00,2024-09-30 09:00:09.495183+00:00,system,system,False


In [28]:
filtered = df_contract[df_contract['iId'] == 13352389]
filtered.head()

,iId,sTpaCode,iDealerId,iFulfillmentDealerId,sContractNumber,sRefNumber,sFormNumber,sContractStatusCode,sContractStatusDesc,sRegisterNumber,sContractPaymentMethod,sDealNumber,sStockNumber,sFandIManagerFistName,sFandIManagerLastName,sVIN,sVehicleClass,sTierCode,sNewUsed,iVehicleYear,sMake,sModel,FuelType,Cyl,DRW,BaseMonths,BaseMiles,CorrosionMonths,CorrosionMiles,EmissionMonths,EmissionMiles,PowertrainMonths,PowertrainMiles,RustMonths,RustMiles,cVehiclePurchasePrice,sFinanceTypeCode,cFinancedAmount,iFinanceTermMonth,cMsrpnada,cMonthlyPayment,cApr,sBundleNumber,sAlternatenumber,iAgentId,sAgentNumber,sAgentName,iLenderId,sLenderNumber,sLenderName,sLenderAddress1,sLenderAddress2,sLenderCity,sLenderState,sLenderZipCode,dSaleDate,dInServiceDate,dEffectiveDate,dExpirationDate,dActivationDate,dBillDate,dtContractCancelledBilled,iSaleOdometer,iEffectiveOdometer,iExpirationOdometer,iStartingOdometer,iEndingOdometer,iServiceInterval,sCustFirstName,sCustLastName,sCustMiddleName,sCustAddress1,sCustAddress2,sCustCity,sCustState,sCustZipCode,sCustPrimaryPhone,sCustSecondaryPhone,sCustEmail,cNetRate,cNetTax1,cNetTax2,cOverRemit,cCurrentBalance,cDealerRemit,cCalculatedDealerPack,cDealerCost,cRetailRate,cRetailTax1,cRetailTax2,cRetailCalculated,bFunded,BalloonAmount,sProductCode,iCoverageId,sCoverageCode,sCoverageDesc,iContractTermMonth,iContractTermMiles,iTermid,sInsurerCode,CoverageGroupCode,CoverageGroupDesc,ReportGroupCode,ReportGroupDesc,OwnerId,OwnerType,RemittancePayeeId,RemittancePayeeType,cDeductibleAmount,iDeductibleType,cDeductibleReducingAmount,sDeductibleTypeCode,DeductibleUniqueId,Ratebookid,RatebookCode,RatebookDesc,sCancellationReason,dCancelDate,dCancelEffectDate,iCancelOdom,cCancelNetRate,cCancelDealerRemit,cCancelRetailRate,nCancellationFactor,cCancellationFee,dCancellationExtractedDate,dCancellationProcessedDate,sCancellationPayeeType,iCancellationPayeeId,sCancellationPayeeNumber,sCancellationPayeeName,sCancellationPayMethod,Balance,cFinancingCancelFee,DealerFee,sCancellationPayeeCountry,sCancellationPayeeAddress1,sCancellationPayeeAddress2,sCancellationPayeeZipCode,sCancellationPayeeState,sCancellationPayeeCity,cNSFFee,cLateCharge,cNumberOfPayments,cPaidClaimDeductOrig,cPaidClaimDeductAdj,cRetailRefundFromTpaOrig,iTransferCount,cTransferFee,dtTransferEffectiveDate,iTransferMileage,sSource,sCustAltContactName,sCoBuyerAddress,sCoBuyerCity,sCoBuyerState,sCoBuyerZipCode,sEarningCurveName,sDevelopmentType,iEarningMainId,iInsertedUserId,iUpdatedUserId,dtInserted,dtUpdated,sCustBirthDate,sCoBuyerBirthDate,sReasonCode,iEarningMainId2,iEarningMainId3,dRatingDate
39151,13352389,ENDR,174717,NaN,EVPU2007699,NaN,VSCFL-01D-SPM-2012,E,Expired,R000012051,ViaRegister,NaN,NaN,NaN,NaN,4JGBB86E79A514600,8,NaN,U,2009,MERCEDES-BENZ,M-Class ML350 4MATIC,Gasoline,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0.00,NaN,0.00,0,0.00,0.00,0.00,NaN,NaN,3693,VPA,VPA-Endurance Direct,68411.00,Paylink,Paylink,150 N. WACKER DRIVE,SUITE 2700,CHICAGO,IL,60606,2015-05-07,2015-05-06,2015-06-06,2018-06-06,2015-06-19,2015-05-01,NaN,88985,89985,119985,0,999999,0,Lance,Authelet,,6622 Marry Vale Ln,NaN,Port Orange,FL,32128,3862144755,0.00,NaN,3296.74,0.00,0.00,0.00,0.00,3296.74,0.00,3296.74,4896.75,0.00,0.00,NaN,False,NaN,VSC,8409,7 SUPREME USED FL - SHELL,VPP Supreme,36,30000,245,WIC,DIRECTV,Direct VSC Non-Wrap,VSC,VSC,2276,Company,2276,Company,200.00,1,0.00,PerRepair,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONV,NaN,NaN,NaN,NaN,NaN,AmTrust 36 month Telemarketing Used,Monthly,1310.00,57535,57535.00,2020-08-30 05:08:56.657000,2023-03-22 05:08:52.267000,NaN,NaN,NaN,1310.00,NaN,NaN


In [29]:
df_contract.head()

,iId,sTpaCode,iDealerId,iFulfillmentDealerId,sContractNumber,sRefNumber,sFormNumber,sContractStatusCode,sContractStatusDesc,sRegisterNumber,sContractPaymentMethod,sDealNumber,sStockNumber,sFandIManagerFistName,sFandIManagerLastName,sVIN,sVehicleClass,sTierCode,sNewUsed,iVehicleYear,sMake,sModel,FuelType,Cyl,DRW,BaseMonths,BaseMiles,CorrosionMonths,CorrosionMiles,EmissionMonths,EmissionMiles,PowertrainMonths,PowertrainMiles,RustMonths,RustMiles,cVehiclePurchasePrice,sFinanceTypeCode,cFinancedAmount,iFinanceTermMonth,cMsrpnada,cMonthlyPayment,cApr,sBundleNumber,sAlternatenumber,iAgentId,sAgentNumber,sAgentName,iLenderId,sLenderNumber,sLenderName,sLenderAddress1,sLenderAddress2,sLenderCity,sLenderState,sLenderZipCode,dSaleDate,dInServiceDate,dEffectiveDate,dExpirationDate,dActivationDate,dBillDate,dtContractCancelledBilled,iSaleOdometer,iEffectiveOdometer,iExpirationOdometer,iStartingOdometer,iEndingOdometer,iServiceInterval,sCustFirstName,sCustLastName,sCustMiddleName,sCustAddress1,sCustAddress2,sCustCity,sCustState,sCustZipCode,sCustPrimaryPhone,sCustSecondaryPhone,sCustEmail,cNetRate,cNetTax1,cNetTax2,cOverRemit,cCurrentBalance,cDealerRemit,cCalculatedDealerPack,cDealerCost,cRetailRate,cRetailTax1,cRetailTax2,cRetailCalculated,bFunded,BalloonAmount,sProductCode,iCoverageId,sCoverageCode,sCoverageDesc,iContractTermMonth,iContractTermMiles,iTermid,sInsurerCode,CoverageGroupCode,CoverageGroupDesc,ReportGroupCode,ReportGroupDesc,OwnerId,OwnerType,RemittancePayeeId,RemittancePayeeType,cDeductibleAmount,iDeductibleType,cDeductibleReducingAmount,sDeductibleTypeCode,DeductibleUniqueId,Ratebookid,RatebookCode,RatebookDesc,sCancellationReason,dCancelDate,dCancelEffectDate,iCancelOdom,cCancelNetRate,cCancelDealerRemit,cCancelRetailRate,nCancellationFactor,cCancellationFee,dCancellationExtractedDate,dCancellationProcessedDate,sCancellationPayeeType,iCancellationPayeeId,sCancellationPayeeNumber,sCancellationPayeeName,sCancellationPayMethod,Balance,cFinancingCancelFee,DealerFee,sCancellationPayeeCountry,sCancellationPayeeAddress1,sCancellationPayeeAddress2,sCancellationPayeeZipCode,sCancellationPayeeState,sCancellationPayeeCity,cNSFFee,cLateCharge,cNumberOfPayments,cPaidClaimDeductOrig,cPaidClaimDeductAdj,cRetailRefundFromTpaOrig,iTransferCount,cTransferFee,dtTransferEffectiveDate,iTransferMileage,sSource,sCustAltContactName,sCoBuyerAddress,sCoBuyerCity,sCoBuyerState,sCoBuyerZipCode,sEarningCurveName,sDevelopmentType,iEarningMainId,iInsertedUserId,iUpdatedUserId,dtInserted,dtUpdated,sCustBirthDate,sCoBuyerBirthDate,sReasonCode,iEarningMainId2,iEarningMainId3,dRatingDate
0,13313238,ENDR,174718,NaN,EVPN1000001,NaN,NaN,R,Rejected,R000000056,ViaRegister,NaN,NaN,NaN,NaN,3FADP4BJ1BM232560,2,NaN,N,2011,FORD,Fiesta SE,Gasoline,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0.00,NaN,0.00,0,0.00,0.00,0.00,NaN,NaN,3693,VPA,VPA-Endurance Direct,71698.00,EWS,EWS,NaN,NaN,NaN,,NaN,2012-09-24,2012-09-24,2012-10-24,2017-10-24,NaN,NaN,NaN,21316,22316,100000,0,999999,0,Nathaniel,Szkil,,2720 Odlum Dr,NaN,Schaumburg,IL,60194,3095331459,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2784.00,0.00,0.00,NaN,False,NaN,VSC,8406,7 SUPREME NEW - SHELL,VPP Supreme,60,100000,243,WIC,DIRECTV,Direct VSC Non-Wrap,VSC,VSC,2276,Company,2276,Company,100.00,1,0.00,PerRepair,2,0,NaN,NaN,NaN,2012-10-11,2012-10-11,0.00,684.00,NaN,2784.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57535,57535.00,2020-08-30 05:08:56.657000,2023-03-22 05:08:26.227000,NaN,NaN,NaN,NaN,NaN,NaN
1,13313239,ENDR,174718,NaN,EVPN1000002,NaN,VSC-01D-EDS-2012,C,Cancelled,R000000056,ViaRegister,NaN,NaN,NaN,NaN,JHMGE8G58BC019508,1,NaN,N,2011,HONDA,Fit Sport,Gasoline,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0.00,NaN,0.00,0,0.00,0.00,0.00,NaN,NaN,3693,VPA,VPA-Endurance Direct,71698.00,EWS,EWS,NaN,NaN,NaN,,NaN,2012-09-24,2012-09-24,2012-10-24,2017-10-24,2012-11-17,2012-11-17,2013-01-24,26817,27817,100000,0,999999,0,Calvin,Stoeck,,1612 E Hou

In [30]:
df_entity = df_entity.rename(columns={'sMailingAddress1':'entityMailingAddress1',
                                      'sMailingAddress2':'entityMailingAddress2', 
                                      'sBillingAddress1':'entityBillingAddress1', 
                                      'sBillingAddress2':'entityBillingAddress2',
                                      'sMailingZipCode': 'entityMailingZipcode',
                                      'sBillingZipCode': 'entityBillingZipcode',
                                     'sMailingCity': 'entityMailingCity',
                                     'sBillingCity': 'entityBillingCity',
                                     'sMailingState': 'entityMailingState',
                                     'sBillingState': 'entityBillingState'})

In [31]:
df_entity['inspectionCompanyName'] = 'n/a'

In [32]:
df_entity.sort_values(by=['inspectionCompanyName'])

,iId,sTpaCode,sServicerNumber,sServicerName,sServicerStatus,sContactFirstName,sContactLastName,sContactMiddleName,entityBillingAddress1,entityBillingAddress2,entityBillingCity,entityBillingState,entityBillingZipcode,iBillingCountryId,entityMailingAddress1,entityMailingAddress2,entityMailingCity,entityMailingState,entityMailingZipcode,iMailingCountryId,sPhoneTypeCode1,sPhoneNumber1,sPhoneExt1,sPhoneTypeCode2,sPhoneNumber2,sPhoneExt2,sEmail,cLaborRate,nPartTax1,nPartTax2,nLaborTax1,nLaborTax2,sImageLogoUrl,sSmtpUser,sSmtpPwd,sSmtpHost,iSmtpPort,sVehicleCategoryType,iInsertedUserId,dtInsertedTime,iUpdatedUserId,dtUpdatedTime,sWebSiteUrl,iServicerTypeId,sClaimPayMethod,sRefundPayMethod,sCreditCardNotifyMethod,sDispatchMethod,sAccountingNumber,sStateLicense,btSelfServicingDealer,sAccountingNumber2,iCurrencyId,btNationalServiceProvider,dContractDate,dContractExpireDate,iServiceRecallPeriod,sServiceRecallPeriodTerm,iInvoiceSubmission,sInvoiceSubmissionTerm,btPartsVerificationRequired,btContracted,btSealedSystemsRepair,iCustomerSatisfaction,iRating,sFacilityType,tOpenSunday,tOpenMonday,tOpenTuesday,tOpenWednesday,tOpenThursday,tOpenFriday,tOpenSaturday,tCloseSunday,tCloseMonday,tCloseTuesday,tCloseWednesday,tCloseThursday,tCloseFriday,tCloseSaturday,sTimeZone,fLatitude,fLongitude,sPaymentFax,sPaymentEmail,iInspectionCompanyId,sServiceCategory,sFedTaxId,inspectionCompanyName
0,501652,ENDR,SC000007,yoder's,A,monica,NaN,,i74 and school st,NaN,carlock,IL,61725,1,i74 and school st,NaN,carlock,IL,61725,1,1,3093764221,NaN,5.00,NaN,NaN,NaN,89.95,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,CAR,57535,2020-01-09 15:31:54.980000,0,2020-01-09 15:31:54.980000,NaN,0,WEX,NaN,None,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,0,NaN,0,NaN,False,False,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RepairFacility,NaN,n/a
65996,612485,ENDR,SR10066757,BOUBIN AUTOMOTIVE,A,DAN,NaN,,799 BLAIRS FERRY RD NE,NaN,CEDAR RAPIDS,IA,52402,1,799 BLAIRS FERRY RD NE,NaN,CEDAR RAPIDS,IA,52402,1,1,3193950030,NaN,5.00,NaN,NaN,NaN,110.00,0.07,0.00,0.07,0.00,NaN,NaN,NaN,NaN,NaN,CAR,95864,2022-02-25 13:55:55.923000,108412,2023-11-30 08:59:33.113000,NaN,0,WEX,NaN,None,,NaN,NaN,False,NaN,NaN,False,NaN,NaN,0,NaN,0,NaN,False,False,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.03,-91.65,3193959052,SHOP4@BOUBIN.COM,NaN,RepairFacility,BgzW9/wb/Sk=,n/a
65995,612482,ENDR,SR10066754,FIRESTONE,A,GENERO,NaN,,5600 SOUTH LA GRANGE RD,NaN,LA GRANGE,IL,60525,1,5600 SOUTH LA GRANGE RD,NaN,LA GRANGE,IL,60525,1,1,7083527505,NaN,5.00,NaN,NaN,NaN,165.00,0.10,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,CAR,95862,2022-02-25 13:51:26.510000,86746,2023-05-17 14:21:24.750000,NaN,0,WEX,NaN,None,,NaN,NaN,False,NaN,NaN,False,NaN,NaN,0,NaN,0,NaN,False,False,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.79,-87.87,7083521740,NaN,NaN,RepairFacility,NaN,n/a
65994,612480,ENDR,SR10066752,FIRESTONE,A,HEATHER,NaN,,702 BALTIMORE PIKE,NaN,BEL AIR,MD,21014,1,702 BALTIMORE PIKE,NaN,BEL AIR,MD,21014,1,1,4108791444,NaN,5.00,NaN,NaN,NaN,152.00,0.06,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,CAR,86753,2022-02-25 13:45:45.190000,116548,2024-05-03 13:45:15.490000,NaN,0,WEX,NaN,None,,NaN,NaN,False,NaN,NaN,False,NaN,NaN,0,NaN,0,NaN,False,False,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.52,-76.36,4108936932,heatherappleby1980@gmail.com,NaN,RepairFacility,NaN,n/a
65993,612478,ENDR,SR10066750,EXPRESS OIL CHANGE & TIREENGINEERS,A,PAUL,NaN,,4640 HWY 90,NaN,MILTON,FL,32571,1,4640 HWY 90,NaN,MILTON,FL,32571,1,1,8507368044,NaN,5.00,NaN,NaN,NaN,150.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,CAR,62700,2022-02-25 13:38:21.283000,129130,2024-09-24 09:30:03.423000,NaN,0,WEX,NaN,None,,NaN,NaN,False,NaN,NaN,False,NaN,NaN,0,NaN,0,NaN,False,False,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.60,-87.13,NaN,paultoddpace@gmail.com,NaN,NaN,NaN,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [33]:
df_entity.head()

,iId,sTpaCode,sServicerNumber,sServicerName,sServicerStatus,sContactFirstName,sContactLastName,sContactMiddleName,entityBillingAddress1,entityBillingAddress2,entityBillingCity,entityBillingState,entityBillingZipcode,iBillingCountryId,entityMailingAddress1,entityMailingAddress2,entityMailingCity,entityMailingState,entityMailingZipcode,iMailingCountryId,sPhoneTypeCode1,sPhoneNumber1,sPhoneExt1,sPhoneTypeCode2,sPhoneNumber2,sPhoneExt2,sEmail,cLaborRate,nPartTax1,nPartTax2,nLaborTax1,nLaborTax2,sImageLogoUrl,sSmtpUser,sSmtpPwd,sSmtpHost,iSmtpPort,sVehicleCategoryType,iInsertedUserId,dtInsertedTime,iUpdatedUserId,dtUpdatedTime,sWebSiteUrl,iServicerTypeId,sClaimPayMethod,sRefundPayMethod,sCreditCardNotifyMethod,sDispatchMethod,sAccountingNumber,sStateLicense,btSelfServicingDealer,sAccountingNumber2,iCurrencyId,btNationalServiceProvider,dContractDate,dContractExpireDate,iServiceRecallPeriod,sServiceRecallPeriodTerm,iInvoiceSubmission,sInvoiceSubmissionTerm,btPartsVerificationRequired,btContracted,btSealedSystemsRepair,iCustomerSatisfaction,iRating,sFacilityType,tOpenSunday,tOpenMonday,tOpenTuesday,tOpenWednesday,tOpenThursday,tOpenFriday,tOpenSaturday,tCloseSunday,tCloseMonday,tCloseTuesday,tCloseWednesday,tCloseThursday,tCloseFriday,tCloseSaturday,sTimeZone,fLatitude,fLongitude,sPaymentFax,sPaymentEmail,iInspectionCompanyId,sServiceCategory,sFedTaxId,inspectionCompanyName
0,501652,ENDR,SC000007,yoder's,A,monica,NaN,,i74 and school st,NaN,carlock,IL,61725,1,i74 and school st,NaN,carlock,IL,61725,1,1,3093764221,NaN,5.00,NaN,NaN,NaN,89.95,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,CAR,57535,2020-01-09 15:31:54.980000,0,2020-01-09 15:31:54.980000,NaN,0,WEX,NaN,None,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,0,NaN,0,NaN,False,False,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RepairFacility,NaN,n/a
1,501653,ENDR,SC000043,RUSNAK PASADENA,A,ASH $265.00,NaN,,337 W COLORADO BLVD,NaN,PASADENA,CA,91105,1,337 W COLORADO BLVD,NaN,PASADENA,CA,91105,1,1,6262292744,NaN,5.00,6262292855.00,NaN,NaN,175.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,CAR,57535,2020-01-09 15:31:54.980000,129128,2023-10-31 12:06:00.717000,NaN,0,WEX,NaN,None,,NaN,NaN,False,NaN,NaN,False,NaN,NaN,0,NaN,0,NaN,False,False,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.15,-118.16,NaN,alooper@rusnakautogroup.com,NaN,RepairFacility,NaN,n/a
2,501654,ENDR,SC000008,elite transmissions,A,JOHNNY,NaN,,1949 CLEARING CT,NaN,NEW LENOX,IL,60451,1,1949 CLEARING CT,NaN,NEW LENOX,IL,60451,1,1,7083719188,NaN,5.00,NaN,NaN,NaN,139.99,0.10,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,CAR,57535,2020-01-09 15:31:54.980000,119520,2024-07-31 08:12:43.123000,NaN,0,WEX,NaN,None,,NaN,NaN,False,NaN,NaN,False,NaN,NaN,0,NaN,0,NaN,False,False,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.49,-87.93,7083717612,elitetranz2014@gmail.com,NaN,RepairFacility,qwS9ozR4/9q67X0Xm9FwwuS6hT+Rfn8s,n/a
3,501655,ENDR,SC000044,Ray Chevrolet,A,TJ 199.00,NaN,,39 N. ROUTE 12,NaN,FOX LAKE,IL,60020,1,39 N. ROUTE 12,NaN,FOX LAKE,IL,60020,1,1,8475873300,NaN,5.00,8475873898.00,NaN,cfreibauer@raychevrolet.com,179.64,0.08,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,CAR,57535,2020-01-09 15:31:54.980000,148456,2024-09-20 15:17:55.513000,NaN,0,WEX,NaN,None,,NaN,NaN,False,NaN,NaN,False,NaN,NaN,0,NaN,0,NaN,False,False,False,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.39,-88.18,8475873898,tburke@raychevrolet.com,NaN,RepairFacility,6sd39obg+Xxr5/zA3xSPAQLKDxs7Jad9pwAKg7P0uMODqTHfaNso3f6+UNkbJO/L02YcraA2P/CvMRxLiH8H0XPtWt2RaweNC+lhCJpRIMomRnWzaqiOFehg4GN/zYztP5SP03WUVV1Jsl/v/gBkv9jurIIno18bCcLm7i06KG9PHuIifnS8xZtOzqZuI9zCwr7U7H0a/2KXjjNT/rq/uzhuQEwR+pUdmQq0m170npqn8BgdU5u/uGnGTSwNFij,n/a
4,501656,ENDR,SC000102,VINNYS AUTOMOTIVE,A,DOMINIC,NaN,,1575 GLENDALE AVE UNIT C,NaN,SPARKS,NV,89431,1,1575 GLENDALE AVE UNIT C,NaN,SPARKS,NV,89431,1,1,7753585255,NaN,5.00,7753585255.00,NaN,NaN,108.45,0.08,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,CAR,57535,2

## Data Load

In [34]:
graph = Neo4jInstance(neo_info['url'],neo_info['user'],neo_info['password'])

### Graph constraints

In [ ]:
# Create indexes and constraints BEFORE loading the data

In [ ]:
graph.execute_write_queries(pre_load_queries,database=neo_info['database'])

### Node Dataframes

In [ ]:
# all dataframes need to be prepared with all the properties the node needs
# the constraints will take care of avoiding duplicate data being loaded, but clean your dataframe as much as possible

In [39]:
nodes_data = {}

#### Address

In [78]:

nodes_data[node_labels[0]]  = df_address

In [49]:
df_agents = pd.concat([
    df_dealer_hd [['sAgentNumber', 'sAgentName']],
    df_contract [['sAgentNumber', 'sAgentName']]
]) 

df_agents.drop_duplicates(inplace=True)

In [50]:
df_agents['sAgentNumber'] = df_agents['sAgentNumber'].str.strip()
df_agents['sAgentName'] = df_agents['sAgentName'].str.strip()

In [51]:
df_agents.head()

,sAgentNumber,sAgentName
0,A000,EDS House Account - No NCB
1,A139,Pinnacle Financial Group Agency
4,A114,Dealer's Choice Inc
6,A127,Progressive Dealer Solutions LLC
7,A145,Complete Auto Protection LLC


In [52]:
nodes_data[node_labels[1]]  = df_agents

In [114]:
df_cities = pd.concat([
    df_vin[['SellerCity']],
    df_claim_pay[['sPayeeCity']],
    df_dealer_hd[['dealerBillingCity']],
    df_dealer_hd[['dealerMailingCity']],
    df_entity[['entityBillingCity']],
    df_entity[['entityMailingCity']],
    df_dealer_gps[['groupBillingCity']],
    df_contract[['sCustCity']]
    ])


In [54]:
df_cities = df_cities.stack(dropna = True).to_frame()
df_cities = df_cities.rename(columns={0: 'cityName'})
df_cities['cityName'] = df_cities['cityName'].str.lower()
df_cities = df_cities.drop_duplicates()

In [55]:
nodes_data[node_labels[2]]  = df_cities

In [56]:
df_claims = df_claim_hd[['iId', 'sClaimNumber', 'sClaimStatusDesc',  'iServiceOdom']]

In [57]:
nodes_data[node_labels[3]]  = df_claims

In [204]:
df_dealer_co = df_dealer_hd[['sInspectionCompanyNumber', 'sInspectionCompanyName']]
df_dealer_co = df_dealer_co.dropna()

df_entity_co = df_entity[['iInspectionCompanyId', 'inspectionCompanyName']]
df_entity_co = df_entity_co.dropna()

df_dealer_co = df_dealer_co.rename(columns={'sInspectionCompanyNumber':'companyNumber', 'sInspectionCompanyName': 'companyName'})
df_entity_co = df_entity_co.rename(columns={'iInspectionCompanyId':'companyNumber', 'inspectionCompanyName': 'companyName'})

In [205]:
df_companies = pd.concat([
    df_entity_co,
    df_dealer_co
])

In [207]:
df_companies = df_companies.astype(str)
df_companies['companyNumber'] = df_companies['companyNumber'].str.replace('.0', '', regex=False)

In [212]:
nodes_data[node_labels[4]]  = df_companies

In [61]:
df_components = df_claim_det[['iClaimCompontentId', 'sDetailStatusDesc', 'sDetailTypeDescription', 'sReasonDescription']]

In [62]:
df_components['sReasonDescription'] = df_components['sReasonDescription'].fillna('')

In [63]:
nodes_data[node_labels[5]]  = df_components

In [64]:
df_contracts = df_contract[['iId', 'sContractNumber', 'sContractStatusDesc', 'sProductCode', 'iContractTermMonth', 'iContractTermMiles']]

In [65]:
nodes_data[node_labels[6]]  = df_contracts

In [66]:
df_coverage = df_contract[['iId', 'sCoverageDesc', 'CoverageGroupDesc', 'cDeductibleAmount', 'iDeductibleType']]

In [67]:
nodes_data[node_labels[7]]  = df_coverage

#### Customer

In [186]:
nodes_data[node_labels[1]]  = df_customers

#### Email

In [76]:
nodes_data[node_labels[2]]  = df_emails

### Create Nodes

In [ ]:
for node in nodes_data:
    rows = nodes_data[node].shape[0]
    if rows > 0:
        partitions, parallel = get_parallel_info(rows)
        print(f'Loading node {node} with {rows} records')
        result = graph.execute_write_query_with_data(node_queries[node],nodes_data[node],
                                                 database=neo_info['database'],partitions=partitions,
                                                 parallel=parallel
                                                )
        print(f'Sucessfully loaded node {node} with the following results: {result}')

In [ ]:
# If you need to create a single node

nodes = ['Address']
for node in nodes:
    rows = nodes_data[node].shape[0]
    partitions, parallel = get_parallel_info(rows)
    print(f'Loading node {node} with {rows} records')
    result = graph.execute_write_query_with_data(node_queries[node],nodes_data[node],
                                                         database=neo_info['database'],
                                                         partitions=partitions,
                                                         parallel=parallel
                                                        )
    print(f'Sucessfully loaded {node} with the following results: {result}')

#### Node Quality Test

In [ ]:
nodes_data_size = []
for label, data in nodes_data.items():
    nodes_data_size.append({'nodeLabel':label,'number_rows':data.shape[0]})
nodes_to_load = pd.DataFrame(nodes_data_size)
nodes_loaded = graph.get_node_label_freq()
nodes_qa = pd.merge(nodes_to_load,nodes_loaded,how='left',on='nodeLabel')
nodes_qa['diff'] = nodes_qa['number_rows'] - nodes_qa['frequency']
nodes_qa.sort_values(['frequency'],ascending=False)

### Relationship Dataframes

In [1]:
# The relationship dataframes will consist of the source and target id propety, plus any other value
# you want to set as a property in the relationship

In [ ]:
rela_types

In [40]:
relationships_data = {}

In [242]:
# Claim ASSIGNED_TO User
df_assigned = df_claim_hd[['iId', 'iAssignedToId', 'dtInserted', 'dtUpdated']]

In [243]:
df_assigned = df_assigned.dropna(subset={'iAssignedToId'})
df_assigned['iId'] = df_assigned['iId'].astype(int)
df_assigned['iAssignedToId'] = df_assigned['iAssignedToId'].astype(str)

In [244]:
df_assigned['iAssignedToId'] = df_assigned['iAssignedToId'].str.replace('.0', '', regex=False)
df_assigned['dtInserted'] = df_assigned['dtInserted'].str[:10]
df_assigned['dtUpdated'] = df_assigned['dtUpdated'].str[:10]

In [245]:
fil = df_assigned[df_assigned['iAssignedToId'] != '0']
fil.head()

,iId,iAssignedToId,dtInserted,dtUpdated
8831,4214505,64098,2020-08-30,2020-09-02
23363,4229037,116551,2020-08-30,2023-09-12
38474,4244148,64097,2020-08-30,2020-10-19
45763,4251437,69585,2020-08-30,2021-08-26
46124,4251798,64089,2020-08-30,2020-11-04


In [246]:
relationships_data[rela_types[0]]  = df_assigned

In [247]:
# Dealer BELONGS_TO Group

df_belongs = df_dealer_lnk[['iDealerId', 'iDealerGroupId']]
df_belongs['iDealerGroupId'] = df_belongs['iDealerGroupId'].astype(str)
df_belongs['iDealerGroupId'] = df_belongs['iDealerGroupId'].str.replace('.0', '', regex=False)

In [248]:
relationships_data[rela_types[1]]  = df_belongs

In [249]:
# Dealer BROUGHT_BY Agent
df_brought = df_dealer_hd[['sAgentNumber', 'iId']]

In [250]:
relationships_data[rela_types[2]]  = df_brought

In [450]:
# Payment for Payee
df_payments = df_claim_det[['iClaimPayeeId', 'iId', 'dApprovedToPayDate', 'dtPaymentProcessed']]
df_payments = df_payments.rename(columns={'iId': 'paymentId'})
df_payments['dApprovedToPayDate'] = df_payments['dApprovedToPayDate'].fillna('')
df_payments['dtPaymentProcessed'] = df_payments['dtPaymentProcessed'].fillna('')


df_payee = df_claim_pay[['iId', 'sPayeeNumber']]
df_payee['dApprovedToPayDate1'] = ''
df_payee['dtPaymentProcessed1'] = ''

df_for = pd.concat([
    df_payments,
    df_payee
],axis=1)

In [454]:
df_for['dApprovedToPayDate'] = df_for.dApprovedToPayDate.combine_first(df_for.dApprovedToPayDate1)
df_for['dtPaymentProcessed'] = df_for.dtPaymentProcessed.combine_first(df_for.dtPaymentProcessed1)

In [456]:
df_for['dApprovedToPayDate'] = df_for['dApprovedToPayDate'].str[:10]
df_for['dtPaymentProcessed'] = df_for['dtPaymentProcessed'].str[:10]

In [457]:
df_for.head()

,iClaimPayeeId,paymentId,dApprovedToPayDate,dtPaymentProcessed,iId,sPayeeNumber,dApprovedToPayDate1,dtPaymentProcessed1
0,4300115,10573718,,,4249148.00,CENTRO,,
1,4300115,10573719,,,4249149.00,SC010663,,
2,4300115,10573720,,,4249150.00,SC002279,,
3,4300115,10573721,,,4249151.00,262,,
4,4350673,10573722,,2016-08-26,4249152.00,CENTRO,,


In [458]:
relationships_data[rela_types[3]]  = df_for

In [254]:
# Contract FOR_VEHICLE
df_for_v = df_contract[['iId', 'sVIN', 'dSaleDate', 'iSaleOdometer', 'iExpirationOdometer']]

In [466]:
df_for_v.dtypes

iId                     int64
sVIN                   object
dSaleDate              object
iSaleOdometer           int64
iExpirationOdometer     int64
dtype: object

In [255]:
relationships_data[rela_types[4]]  = df_for_v

#### HAS_ADDRESS

In [102]:
relationships_data[rela_types[0]]  = df_cust_adr

In [283]:
df_has_contract = df_contract[['iId', 'sContractNumber', 'dSaleDate','dExpirationDate','dActivationDate']]

df_has_contract['dSaleDate'] = df_has_contract['dSaleDate'].fillna('')
df_has_contract['dExpirationDate'] = df_has_contract['dExpirationDate'].fillna('')
df_has_contract['dActivationDate'] = df_has_contract['dActivationDate'].fillna('')

In [284]:
relationships_data[rela_types[12]]  = df_has_contract

In [285]:
df_details = df_contract[['sVIN']]

In [286]:
relationships_data[rela_types[13]]  = df_details

In [287]:
# Seller HAS_DEAL Deal

df_has_deal = df_vin[['SellerName', 'Vin']]
df_has_deal['SellerName'] = df_has_deal['SellerName'].str.strip()
df_has_deal['SellerName'] = df_has_deal['SellerName'].dropna()

In [288]:
relationships_data[rela_types[14]]  = df_has_deal

#### HAS_EMAIL

In [290]:
relationships_data[rela_types[1]]  = df_cust_email

In [307]:
#Component HAS_LOSS

df_has_loss = df_claim_det[['iLossCodeId', 'iClaimCompontentId']]

In [308]:
relationships_data[rela_types[24]]  = df_has_loss

In [309]:
# Claim HAS_NOTE

df_has_note = df_claim_nts[['iClaimId', 'iId', 'dtInsertedTime', 'dtUpdatedTime']]
df_has_note['dtInsertedTime'] = df_has_note['dtInsertedTime'].fillna('')
df_has_note['dtUpdatedTime'] = df_has_note['dtUpdatedTime'].fillna('')
df_has_note['dtInsertedTime'] = df_has_note['dtInsertedTime'].str[:10]
df_has_note['dtUpdatedTime'] = df_has_note['dtUpdatedTime'].str[:10]

In [310]:
relationships_data[rela_types[25]]  = df_has_note

In [311]:
# Customer

df_cust_ph = df_contract[['sCustPrimaryPhone', 'sContractNumber']]
df_cust_ph2 = df_contract[['sCustSecondaryPhone', 'sContractNumber']]

df_cust_ph = df_cust_ph.rename(columns={'sCustPrimaryPhone': 'phone'})
df_cust_ph2 = df_cust_ph.rename(columns={'sCustSecondaryPhone': 'phone'})

In [312]:
df_cust_ph = pd.concat([
    df_cust_ph,
    df_cust_ph
])

In [313]:
df_cust_ph = df_cust_ph.dropna()

In [314]:
df_cust_ph = df_cust_ph.astype(str)
df_cust_ph['phone'] = df_cust_ph['phone'].str.replace('.0', '', regex=False)
df_cust_ph['phone'] = df_cust_ph['phone'].str.replace(r'\D+', '', regex=True)

In [315]:
relationships_data[rela_types[26]]  = df_cust_ph

In [316]:
# Dealer HAS_PHONE

df_deal_ph = df_dealer_hd[['sDealerBusinessPhone', 'iId']]
df_deal_ph = df_deal_ph.dropna()
df_deal_ph = df_deal_ph.astype(str)
df_deal_ph['sDealerBusinessPhone'] = df_deal_ph['sDealerBusinessPhone'].str.replace('.0', '', regex=False)
df_deal_ph['sDealerBusinessPhone'] = df_deal_ph['sDealerBusinessPhone'].str.replace(r'\D+', '', regex=True)

In [317]:
relationships_data[rela_types[27]]  = df_deal_ph

In [318]:
# Group HAS_PHONE
df_grp_ph = df_dealer_gps[['iId', 'BusinessPhone']]
df_grp_ph = df_grp_ph.dropna()
df_grp_ph = df_grp_ph.astype(str)
df_grp_ph['BusinessPhone'] = df_grp_ph['BusinessPhone'].str.replace('.0', '', regex=False)
df_grp_ph['BusinessPhone'] = df_grp_ph['BusinessPhone'].str.replace(r'\D+', '', regex=True)

In [319]:
relationships_data[rela_types[28]]  = df_grp_ph

In [320]:
# Payee

df_payee_ph = df_claim_pay[['sPayeeNumber', 'sPayeePhoneNumber']]
df_payee_ph = df_payee_ph.dropna()
df_payee_ph = df_payee_ph.astype(str)
df_payee_ph['sPayeePhoneNumber'] = df_payee_ph['sPayeePhoneNumber'].str.replace('.0', '', regex=False)
df_payee_ph['sPayeePhoneNumber'] = df_payee_ph['sPayeePhoneNumber'].str.replace(r'\D+', '', regex=True)

In [321]:
relationships_data[rela_types[29]]  = df_payee_ph

In [322]:
# Servicer

df_svc_ph = df_entity[['sPhoneNumber1', 'iId']]
df_svc_ph2 = df_entity[['sPhoneNumber2', 'iId']]

df_svc_ph = df_svc_ph.rename(columns={'sPhoneNumber1': 'phone'})
df_svc_ph2 = df_svc_ph2.rename(columns={'sPhoneNumber2': 'phone'})

In [323]:
df_svc_ph = pd.concat([
    df_svc_ph,
    df_svc_ph2
])

In [324]:
df_svc_ph = df_svc_ph.dropna()
df_svc_ph = df_svc_ph.astype(str)
df_svc_ph['phone'] = df_svc_ph['phone'].str.replace('.0', '', regex=False)
df_svc_ph['phone'] = df_svc_ph['phone'].str.replace(r'\D+', '', regex=True)

In [325]:
relationships_data[rela_types[30]]  = df_svc_ph

In [326]:
# Seller

df_seller_ph = df_vin[['SellerPhone', 'SellerName']]
df_seller_ph = df_seller_ph.dropna()
df_seller_ph = df_seller_ph.astype(str)
df_seller_ph['SellerPhone'] = df_seller_ph['SellerPhone'].str.replace('.0', '', regex=False)

In [327]:
df_seller_ph['SellerPhone'] = df_seller_ph['SellerPhone'].str.replace(r'\D+', '', regex=True)

In [328]:
relationships_data[rela_types[31]]  = df_seller_ph

In [329]:
# Dealer

df_dealer_web = df_dealer_hd[['sDealerWebsite', 'iId']]
df_dealer_web = df_dealer_web.dropna()

In [330]:
df_dealer_web['sDealerWebsite'] = df_dealer_web['sDealerWebsite'].str.lower()
df_dealer_web['sDealerWebsite'] = df_dealer_web['sDealerWebsite'].str.replace('http://', '', regex=False)
df_dealer_web['sDealerWebsite'] = df_dealer_web['sDealerWebsite'].str.replace('https://', '', regex=False)
df_dealer_web['sDealerWebsite'] = df_dealer_web['sDealerWebsite'].str.replace('www.', '', regex=False)
df_dealer_web['sDealerWebsite'] = df_dealer_web['sDealerWebsite'].str.replace('/', '', regex=False)

In [331]:
relationships_data[rela_types[32]]  = df_dealer_web

In [332]:
# Seller 

df_sell_web = df_vin[['SellerName', 'SellerWebsite']]
df_sell_web = df_sell_web.dropna()

In [333]:
df_sell_web['SellerWebsite'] = df_sell_web['SellerWebsite'].str.lower()
df_sell_web['SellerWebsite'] = df_sell_web['SellerWebsite'].str.replace('http://', '', regex=False)
df_sell_web['SellerWebsite'] = df_sell_web['SellerWebsite'].str.replace('https://', '', regex=False)
df_sell_web['SellerWebsite'] = df_sell_web['SellerWebsite'].str.replace('www.', '', regex=False)
df_sell_web['SellerWebsite'] = df_sell_web['SellerWebsite'].str.replace('/', '', regex=False)

In [334]:
relationships_data[rela_types[33]]  = df_sell_web

In [51]:
# Servicer

df_svc_web = df_entity[['iId', 'sWebSiteUrl']]
df_svc_web = df_svc_web.dropna()

In [52]:
df_svc_web.head()

,iId,sWebSiteUrl


In [337]:
#df_svc_web['sWebSiteUrl'] = df_svc_web['sWebSiteUrl'].str.lower()
#df_svc_web['sWebSiteUrl'] = df_svc_web['sWebSiteUrl'].str.replace('http://', '', regex=False)
##df_svc_web['sWebSiteUrl'] = df_svc_web['sWebSiteUrl'].str.replace('https://', '', regex=False)
#df_svc_web['sWebSiteUrl'] = df_svc_web['sWebSiteUrl'].str.replace('www.', '', regex=False)
#df_svc_web['sWebSiteUrl'] = df_svc_web['sWebSiteUrl'].str.replace('/', '', regex=False)

In [338]:
relationships_data[rela_types[34]]  = df_svc_web

In [339]:
# Dealer
df_dealer_tax = df_dealer_hd[['iId', 'sFedTaxId']]

In [340]:
df_dealer_tax = df_dealer_tax.dropna()
df_dealer_tax['sFedTaxId'] = df_dealer_tax['sFedTaxId'].str.replace(r'\D+', '', regex=True)

In [341]:
relationships_data[rela_types[35]]  = df_dealer_tax

In [342]:
# Servicer

df_svc_tax = df_entity[['iId', 'sFedTaxId']]

In [343]:
df_svc_tax = df_svc_tax.dropna()
df_svc_tax['sFedTaxId'] = df_svc_tax['sFedTaxId'].str.replace(r'\D+', '', regex=True)

In [344]:
relationships_data[rela_types[36]]  = df_svc_tax

In [149]:
# Address IN Zipcode

df_address_zip = pd.concat ([
                        df_claim_pay[['sPayeeAddress1',	'sPayeeAddress2', 'sPayeeZipCode']],
                        df_dealer_hd[['dealerMailingAddress1',	'dealerMailingAddress2', 'dealerBillingAddress1',	'dealerBillingAddress2', 'dealerMailingZipcode', 'dealerBillingZipcode']],
                        df_entity[['entityMailingAddress1','entityMailingAddress2', 'entityBillingAddress1', 'entityBillingAddress2', 'entityMailingZipcode', 'entityBillingZipcode']],
                        df_dealer_gps[['groupBillingAddress1', 'groupBillingAddress2', 'groupBillingZipcode']],
                        df_contract[['sCustAddress1', 'sCustAddress2', 'sCustZipCode']]
                        ],
                        axis = 'columns'
                       )     

In [150]:
df_address_zip = df_address_zip.fillna('')

In [151]:
df_address_zip['payeeAddress']=  df_address_zip['sPayeeAddress1']+' '+df_address_zip['sPayeeAddress2']
df_address_zip['dealerMailingAddress']=  df_address_zip['dealerMailingAddress1']+' '+df_address_zip['dealerMailingAddress2']
df_address_zip['dealerBillingAddress']=  df_address_zip['dealerBillingAddress1']+' '+df_address_zip['dealerBillingAddress2']
df_address_zip['entityMailingAddress']=  df_address_zip['entityMailingAddress1']+' '+df_address_zip['entityMailingAddress2']
df_address_zip['entityBillingAddress']=  df_address_zip['entityBillingAddress1']+' '+df_address_zip['entityBillingAddress2']
df_address_zip['dealerBillingAddress']=  df_address_zip['entityBillingAddress1']+' '+df_address_zip['entityBillingAddress2']
df_address_zip['groupBillingAddress']=  df_address_zip['groupBillingAddress1']+' '+df_address_zip['groupBillingAddress2']
df_address_zip['custAddress']=  df_address_zip['sCustAddress1']+' '+df_address_zip['sCustAddress2']

In [152]:
df_address_zip = df_address_zip.drop(['sPayeeAddress2', 
                     'dealerMailingAddress2', 
                     'dealerBillingAddress2', 
                     'entityMailingAddress2', 
                     'entityBillingAddress2', 
                     'groupBillingAddress2', 
                     'sCustAddress2']
                   ,axis=1)

In [153]:
df_p = df_address_zip[['payeeAddress', 'sPayeeZipCode']]
df_dm = df_address_zip[['dealerMailingAddress', 'dealerMailingZipcode']]
df_db = df_address_zip[['dealerBillingAddress', 'dealerBillingZipcode']]
df_em = df_address_zip[['entityMailingAddress', 'entityMailingZipcode']]
df_eb = df_address_zip[['entityBillingAddress', 'entityBillingZipcode']]
df_gb = df_address_zip[['groupBillingAddress', 'groupBillingZipcode']]
df_cm = df_address_zip[['custAddress', 'sCustZipCode']]

In [154]:
df_p = df_p.rename(columns={'payeeAddress': 'address', 'sPayeeZipCode':'zipcode'})
df_dm = df_dm.rename(columns={'dealerMailingAddress': 'address', 'dealerMailingZipcode':'zipcode'})
df_db = df_db.rename(columns={'dealerBillingAddress': 'address', 'dealerBillingZipcode':'zipcode'})
df_em = df_em.rename(columns={'entityMailingAddress': 'address', 'entityMailingZipcode':'zipcode'})
df_eb = df_eb.rename(columns={'entityBillingAddress': 'address', 'entityBillingZipcode':'zipcode'})
df_gb = df_gb.rename(columns={'groupBillingAddress': 'address', 'groupBillingZipcode':'zipcode'})
df_cm = df_cm.rename(columns={'custAddress': 'address', 'sCustZipCode':'zipcode'})

In [155]:
df_address_zip = pd.concat([
    df_p,
    df_dm,
    df_db,
    df_em,
    df_eb,
    df_gb,
    df_cm
])

In [156]:
df_address_zip.loc[df_address_zip['address'].astype(str).str.len().lt(1),'address']='n/a'

In [159]:
df_address_zip = clean_address(df_address_zip, 'address')

In [160]:
df_address_zip['zipcode'] = df_address_zip['zipcode'].str.pad(width=5, side='left', fillchar='0')

In [161]:
relationships_data[rela_types[37]]  = df_address_zip

In [356]:
# Zipcode IN City
df_zip_city = pd.concat([
    df_vin[['SellerCity', 'SellerZip']],
    df_claim_pay[['sPayeeCity', 'sPayeeZipCode']],
    df_dealer_hd[['dealerBillingCity', 'dealerBillingZipcode']],
    df_dealer_hd[['dealerMailingCity', 'dealerMailingZipcode']],
    df_entity[['entityBillingCity', 'entityBillingZipcode']],
    df_entity[['entityMailingCity', 'entityMailingZipcode']],
    df_dealer_gps[['groupBillingCity', 'groupBillingZipcode']],
    df_contract[['sCustCity', 'sCustZipCode']]
    ],
    axis = 'columns')

df_zip_city = df_zip_city.dropna()

In [357]:
df_s = df_zip_city[['SellerCity', 'SellerZip']]
df_c = df_zip_city[['sPayeeCity', 'sPayeeZipCode']]
df_db2 = df_zip_city[['dealerBillingCity', 'dealerBillingZipcode']]
df_dm = df_zip_city[['dealerMailingCity', 'dealerMailingZipcode']]
df_eb2 = df_zip_city[['entityBillingCity', 'entityBillingZipcode']]
df_em2 = df_zip_city[['entityMailingCity', 'entityMailingZipcode']]
df_dg2 = df_zip_city[['groupBillingCity', 'groupBillingZipcode']]
df_co = df_zip_city[['sCustCity', 'sCustZipCode']]

In [358]:
df_s = df_s.rename(columns={'SellerCity': 'city', 'SellerZip':'zipcode'})
df_c = df_c.rename(columns={'sPayeeCity': 'city', 'sPayeeZipCode':'zipcode'})
df_db2 = df_db2.rename(columns={'dealerBillingCity': 'city', 'dealerBillingZipcode':'zipcode'})
df_dm = df_dm.rename(columns={'dealerMailingCity': 'city', 'dealerMailingZipcode':'zipcode'})
df_eb2 = df_eb2.rename(columns={'entityBillingCity': 'city', 'entityBillingZipcode':'zipcode'})
df_em2 = df_em2.rename(columns={'entityMailingCity': 'city', 'entityMailingZipcode':'zipcode'})
df_dg2 = df_dg2.rename(columns={'groupBillingCity': 'city', 'groupBillingZipcode':'zipcode'})
df_co = df_co.rename(columns={'sCustCity': 'city', 'sCustZipCode':'zipcode'})

In [359]:
df_zip_city = pd.concat([
    df_s,
    df_c,
    df_db2,
    df_dm,
    df_eb2,
    df_em2,
    df_dg2,
    df_co
])

In [360]:
df_zip_city['zipcode'] = df_zip_city['zipcode'].str.pad(width=5, side='left', fillchar='0')
df_zip_city['city'] = df_zip_city['city'].str.lower()

In [490]:
df_zip_city.head()

,city,zipcode
1375,canal fulton,44614
1377,garden grove,92843
1402,jacksonville,32244
1410,golden,80401
1435,overland park,66212


In [491]:
relationships_data[rela_types[38]]  = df_zip_city

In [492]:
# City IN State
df_city_state = pd.concat([
    df_vin[['SellerCity', 'SellerState']],
    df_claim_pay[['sPayeeCity', 'sPayeeState']],
    df_dealer_hd[['dealerBillingCity', 'dealerBillingState']],
    df_dealer_hd[['dealerMailingCity', 'dealerMailingState']],
    df_entity[['entityBillingCity', 'entityBillingState']],
    df_entity[['entityMailingCity', 'entityMailingState']],
    df_dealer_gps[['groupBillingCity', 'groupBillingState']],
    df_contract[['sCustCity', 'sCustState']]
    ],
    axis = 'columns')

df_city_state = df_city_state.dropna()

In [493]:
df_ss = df_city_state[['SellerCity', 'SellerState']]
df_cs = df_city_state[['sPayeeCity', 'sPayeeState']]
df_dbs = df_city_state[['dealerBillingCity', 'dealerBillingState']]
df_dms = df_city_state[['dealerMailingCity', 'dealerMailingState']]
df_ebs = df_city_state[['entityBillingCity', 'entityBillingState']]
df_ems = df_city_state[['entityMailingCity', 'entityMailingState']]
df_dgs = df_city_state[['groupBillingCity', 'groupBillingState']]
df_cos = df_city_state[['sCustCity', 'sCustState']]

In [494]:
df_ss = df_ss.rename(columns={'SellerCity': 'city', 'SellerState':'state'})
df_cs = df_cs.rename(columns={'sPayeeCity': 'city', 'sPayeeState':'state'})
df_dbs = df_dbs.rename(columns={'dealerBillingCity': 'city', 'dealerBillingState':'state'})
df_dms = df_dms.rename(columns={'dealerMailingCity': 'city', 'dealerMailingState':'state'})
df_ebs = df_ebs.rename(columns={'entityBillingCity': 'city', 'entityBillingState':'state'})
df_ems = df_ems.rename(columns={'entityMailingCity': 'city', 'entityMailingState':'state'})
df_dgs = df_dgs.rename(columns={'groupBillingCity': 'city', 'groupBillingState':'state'})
df_cos = df_cos.rename(columns={'sCustCity': 'city', 'sCustState':'state'})

In [495]:
df_city_state = pd.concat([
    df_ss,
    df_cs,
    df_dbs,
    df_dms,
    df_ebs,
    df_ems,
    df_dgs,
    df_cos
])

In [496]:
df_city_state['city'] = df_city_state['city'].str.lower()
df_city_state['state'] = df_city_state['state'].str.lower()

In [497]:
relationships_data[rela_types[39]]  = df_city_state

In [498]:
# Claim INCLUDES Component

df_includes = df_claim_det[['iClaimId', 'iClaimCompontentId', 'dAuthorizedDate', 'dDeniedDate']]
df_includes['dAuthorizedDate'] = df_includes['dAuthorizedDate'].fillna('')
df_includes['dDeniedDate'] = df_includes['dDeniedDate'].fillna('')

In [499]:
relationships_data[rela_types[40]]  = df_includes

In [500]:
# Dealer INSPECTED_BY Company

df_dinspectedby = df_dealer_hd[['sInspectionCompanyNumber', 'iId']]
df_dinspectedby = df_dinspectedby.dropna()

In [502]:
relationships_data[rela_types[41]]  = df_dinspectedby

In [503]:
# Servicer INSPECTED_BY Company

df_sinspectedby = df_entity[['iInspectionCompanyId', 'iId']]
df_sinspectedby = df_sinspectedby.dropna()

In [504]:
df_sinspectedby = df_sinspectedby.astype(str)
df_sinspectedby['iInspectionCompanyId'] = df_sinspectedby['iInspectionCompanyId'].str.replace('.0', '', regex=False)

In [505]:
relationships_data[rela_types[42]]  = df_sinspectedby

In [375]:
# Claim ISSUES_PAYMENT Payment

df_issues_pmt = df_claim_det[['dDeniedDate', 'dApprovedToPayDate', 'iClaimId', 'iId']]
df_issues_pmt['dApprovedToPayDate'] = df_issues_pmt['dApprovedToPayDate'].fillna('')
df_issues_pmt['dDeniedDate'] = df_issues_pmt['dDeniedDate'].fillna('')

In [506]:
relationships_data[rela_types[43]]  = df_issues_pmt

In [128]:
# Payee LIVES_IN Address

df_plives_in = df_claim_pay[['sPayeeAddress1',	'sPayeeAddress2', 'sPayeeNumber']]
df_plives_in['sPayeeAddress2'] = df_plives_in['sPayeeAddress2'].fillna('')
df_plives_in['payeeAddress']=  df_plives_in['sPayeeAddress1']+' '+df_plives_in['sPayeeAddress2']

In [129]:
df_plives_in = clean_address(df_plives_in, 'payeeAddress')

In [130]:
relationships_data[rela_types[44]]  = df_plives_in

In [132]:
# Seller LIVES_IN Address
df_slives_in = df_vin[['SellerName', 'SellerAddress']]
df_slives_in = df_slives_in.dropna()

In [133]:
df_slives_in['SellerAddress'] = df_slives_in['SellerAddress'].fillna('')

In [134]:
df_slives_in = clean_address(df_slives_in, 'SellerAddress')

In [135]:
relationships_data[rela_types[45]]  = df_slives_in

In [385]:
# Component NEEDS Part
df_needs = df_claim_det[['iClaimCompontentId', 'sPartNumber']]
df_needs = df_needs.dropna()

In [509]:
relationships_data[rela_types[46]]  = df_needs

In [387]:
# Part REQUESTS PartQty
df_requests = df_claim_det[['sPartNumber', 'iId', 'dAuthorizedDate', 'dDeniedDate']]
df_requests['dAuthorizedDate'] = df_requests['dAuthorizedDate'].fillna('')
df_requests['dDeniedDate'] = df_requests['dDeniedDate'].fillna('')
df_requests = df_requests.dropna()

In [510]:
relationships_data[rela_types[47]]  = df_requests

In [511]:
# Component REQUESTS Labor

df_lrequests = df_claim_det[['iClaimCompontentId', 'iId', 'dAuthorizedDate', 'dDeniedDate']]
df_lrequests['dAuthorizedDate'] = df_lrequests['dAuthorizedDate'].fillna('')
df_lrequests['dDeniedDate'] = df_lrequests['dDeniedDate'].fillna('')
df_lrequests = df_lrequests.dropna()

relationships_data[rela_types[48]]  = df_lrequests

In [44]:
# Deal RELATED_TO Vehicle

df_relatedto = df_vin[['Vin', 'Date']]
df_relatedto['Date'] = df_relatedto['Date'].str[:10]

In [45]:
df_relatedto.head()

,Vin,Date
0,19UDE2F30GA021226,2024-09-08
1,19UDE2F30LA000497,2024-09-06
2,19UDE2F31HA010902,2024-09-08
3,19UDE2F34HA004415,2024-08-30
4,19UDE2F34JA008924,2024-03-06


In [47]:
relationships_data[rela_types[49]]  = df_relatedto

In [36]:
# Contract HAS_CLAIM Claim
df_has_claim = df_claim_hd[['iContractId', 'iId', 'dClaimDate']]

In [37]:
df_has_claim.head()

,iContractId,iId,dClaimDate
0,13898388,4205674,2015-06-03
1,13871780,4205675,2016-08-22
2,13871780,4205676,2016-05-23
3,13871851,4205677,2016-10-03
4,13871839,4205678,2016-09-27


In [42]:
relationships_data[rela_types[50]]  = df_has_claim

### Create Relationships

In [ ]:
for rela in relationships_data:
    rows = relationships_data[rela].shape[0]
    if rows > 0:
        partitions, parallel = get_parallel_info(rows)
        print(f'Loading relationship {rela} with {rows} rows and {partitions} partitions')
        result = graph.execute_write_query_with_data(rela_queries[rela],relationships_data[rela],
                                                 database=neo_info['database'],partitions=partitions,
                                                 parallel=parallel
                                                )
        print(f'Sucessfully loaded relationship {rela} with the following results: {result}  \n')

In [ ]:
# If you need to load a single relationship

rel = 'CUSTOMER_HAS_ADDRESS'
rows = relationships_data[rel].shape[0]
partitions, parallel = get_parallel_info(rows)
print(f'Loading relationship {rel} with {rows} rows and {partitions} partitions')
result = graph.execute_write_query_with_data(rela_queries[rel],
                                             relationships_data[rel],
                                                 database=neo_info['database'],
                                             partitions=partitions,
                                                 parallel=parallel
                                                )
print(f'Sucessfully loaded relationship {rel} with the following results: {result}')